In [96]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from os.path import exists
from pathlib import Path
from tqdm.notebook import trange, tqdm
from hf import clean_text, replaceWithNull, standard_clean, hard_clean

In [97]:
#The specified url and the letters for our group.
url = "https://en.wikinews.org/wiki/Category:Politics_and_conflicts"
signs = ["H","I","J","K","L","M","N","O","P","R"]

In [98]:
file_exists = exists("../Data/wiki.csv")
if file_exists == False: 
    # We use the libaries request and beautifulsoup.
    website = requests.get(url)
    soup = BeautifulSoup(website.content, 'html.parser')
    data = soup.findAll("div", {"class":"mw-category-group"})

    # Extracting all subcategories of articles that contain our letters.
    _data = []
    for element in data:
        if element.find("h3").contents[0] in signs:
            for href in element.findAll('a', href=True):
                _data.append(href['href'])

    # We now use the links to find the links in subpages.
    _data2 = []
    for element in _data:
        response = requests.get('https://en.wikinews.org/' + element)
        text_in_response = response.text
        soup_a = BeautifulSoup(text_in_response, 'html.parser')
        temp = soup_a.findAll('div', id='mw-pages')
        
        for element in temp:
            temp2 = element.findAll('a')
            for element in temp2:
                _data2.append(element['href'])

                dates = []
    titles = []
    content = []
    
    for element1 in tqdm(_data2):
        response = requests.get('https://en.wikinews.org/' + element1)
        soup_b = BeautifulSoup(response.text, 'html.parser')
        
        #Extract date from text. Some cases does not have a date, if so we insert N/A
        try: # look for HTML element
            dates.append(soup_b.find('strong', class_="published").contents[1])
        except: # add NA if nothing found
            dates.append("N/A")
        
        #Extract title of article
        try:
            titles.append(soup_b.find('h1', class_="firstHeading mw-first-heading").contents[0])
        #titles.append(title)
        except:  
            dates.append("N/A")
            
        #Extract main content from the article
        article_content = ""
        for sub_text in soup_b.find_all('p'):
            sub_content = str(sub_text.text)
            article_content += sub_content
        
        content.append(article_content)
        dfw = pd.DataFrame(list(zip(titles, dates, content)), columns =['Title', 'Date published', 'Article content'])
         
else: 
    file = Path("../Data/wiki.csv")
    dfw = pd.read_csv(file)
    #dfw = pd.DataFrame(list(zip(titles, dates, content)), columns =['Title', 'Date published', 'Article content'])
    dfw.to_csv(file, index=False)   

In [99]:

file = Path("../Data/wiki.csv")
print (dfw.shape)
dfw.to_csv(file, index=False)

# Code for reading the csv file and separating.
data = []

with open (file, 'r', encoding='utf-8') as reader: 
    reader = pd.read_csv(reader, chunksize=1000,quotechar='"', delimiter=',')
    for chunk in (reader): 
        chunk = replaceWithNull(chunk)
        for row_no in tqdm(range(len(chunk.index)), leave=False , position = 0):
            for col_no in range (len(chunk.columns)):
                chunk.iloc[row_no,col_no] = clean_text((chunk.iloc[row_no,col_no]))
        data.append(chunk)

reader.close()
data = pd.concat(data)
# Converting to a pandas dataframe. 
dfw = pd.DataFrame(data)   

dfw.columns = ['Title', 'Date published', 'Article content']

dfw.to_csv(file, index=False)

(1212, 3)


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/212 [00:00<?, ?it/s]